# Prediction process for new utterances.

# Setup the environment
#### 1. In order to do this we need to import the modules of the application

In [2]:
import tensorflow as tf
import os
import data_loader
import model
import main_module
import numpy as np

tf.logging.set_verbosity(tf.logging.ERROR) # removes the warning messeges that pollute the console

##### 2. To set the basis for prediction we need to encode words into vectors, we can't perform computations on vectors.
Moreover, we need the intents and slot types defined as well. These informations are extracted from the dataset on which the model was trained.

The data_loader module loads the word vectors into memory (huge dictionary of words as keys and vectors as values). Along with loading the word vectors it encodes the training and testing data as well. While encoding the data it also extracts the possible intents and slot types and assigns them ids. 

###### This step takes the most time, on my machine it takes about 10 minutes to load the word vectors into memory, about 4 gb with a speed of 6 - 8 mb/s.

In [2]:
data = data_loader.read_datasets()

------------------read datasets begin-------------------
------------------load word2vec begin-------------------
------------------load word2vec end---------------------
------------------read datasets end---------------------


#### 3. Now that we have the data set up, next we need to set the flags for the architecture. 
Tensorflow permits working with flags helping with the processing of command line parameters as well as generally providing a way to better organize the variable parameters to an application.

Some of these flags are computed from the given data, others are assigned by us with values obtianed in the hyperparameter tuning process. 

In [3]:
# load settings
vocab_size, word_emb_size = data['embedding'].shape
_, max_sentence_length = data['x_tr'].shape
intents_number = len(data['intents_dict'])
slots_number = len(data['slots_dict'])

FLAGS = tf.app.flags.FLAGS
tf.app.flags.DEFINE_float("keep_prob", 0.8, "embedding dropout keep rate")
tf.app.flags.DEFINE_integer("hidden_size", 32, "embedding vector size")
tf.app.flags.DEFINE_integer("batch_size", 1, "vocab size of word vectors")
tf.app.flags.DEFINE_integer("num_epochs", 20, "num of epochs")
tf.app.flags.DEFINE_integer("vocab_size", vocab_size, "vocab size of word vectors")
tf.app.flags.DEFINE_integer("max_sentence_length", max_sentence_length, "max number of words in one sentence")
tf.app.flags.DEFINE_integer("intents_nr", intents_number, "intents_number")  #
tf.app.flags.DEFINE_integer("slots_nr", slots_number, "slots_number")  #
tf.app.flags.DEFINE_integer("word_emb_size", word_emb_size, "embedding size of word vectors")
tf.app.flags.DEFINE_boolean("use_embedding", True, "whether to use embedding or not.")
tf.app.flags.DEFINE_float("learning_rate", 0.01, "learning rate")
tf.app.flags.DEFINE_float("margin", 1.0, "ranking loss margin")
tf.app.flags.DEFINE_integer("slot_routing_num", 4, "slot routing num")
tf.app.flags.DEFINE_integer("intent_routing_num", 4, "intent routing num")
tf.app.flags.DEFINE_integer("re_routing_num", 3, "re routing num")
tf.app.flags.DEFINE_integer("intent_output_dim", 64, "intent output dimension")
tf.app.flags.DEFINE_integer("slot_output_dim", 128, "slot output dimension")
tf.app.flags.DEFINE_integer("attention_output_dimenison", 20, "self attention weight hidden units number")
tf.app.flags.DEFINE_float("alpha", 0.001, "coefficient for self attention loss")
tf.app.flags.DEFINE_integer("r", 3, "self attention weight hops")
tf.app.flags.DEFINE_boolean("save_model", False, "save model to disk")
tf.app.flags.DEFINE_boolean("test", False, "Evaluate model on test data")
tf.app.flags.DEFINE_boolean("crossval", False, "Perform k-fold cross validation")
tf.app.flags.DEFINE_integer("n_splits", 3, "Number of cross-validation splits")
tf.app.flags.DEFINE_string("summaries_dir", './logs', "tensorboard summaries")
tf.app.flags.DEFINE_string("ckpt_dir", './saved_models/Scenario0', "check point dir")
tf.app.flags.DEFINE_string("scenario_num", '', "Scenario number")
tf.app.flags.DEFINE_string('f', '', 'kernel')  # required by jupyter notebook 

#### 4. Load the model from the checkpoint directory:

In [7]:
tf.reset_default_graph()
config = tf.ConfigProto()
sess = tf.Session(config=config) 
# Instantiate Model
capsnet = model.capsnet(FLAGS)
if os.path.exists(FLAGS.ckpt_dir):
    print("Restoring Variables from Checkpoint for testing")
    saver = tf.train.Saver()
    saver.restore(sess, tf.train.latest_checkpoint(FLAGS.ckpt_dir))
else:
    print("No trained model exists in checkpoint dir!")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
dim is deprecated, use axis instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Restoring Variables from Checkpoint for testing
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./saved_models/Scenario0\model.ckpt-1
Utterance classified as: scadeIntensitateLumina
Slots:
['O', 'O',

#### 5. Now that we have everything loaded, the only thing left is to input the new data samples to be predicted. 
The next cells have the responsibility to prepare the new data in the right format for prediction.
   
Important guidelines are that the intents and slots should be from the set of predefined ones found in the training data. Moreover, the utterance has must not exceed the length of the longest utterance seen in the data set. In the following we will address this issue.

In [4]:
input_sentence = 'fa mai mica intensitatea iluminatului in sufragerie la 4'

fa mai mica intensitatea iluminatului in sufragerie la 4


#### 6. We can't perform computations on words in text hence we need to transform the input sentence into words vectors, using the same word vectors used to encode the trainind data.

In [5]:
sentence_words = input_sentence.split(' ')
encoded_words = []
for w in sentence_words:
    encoded_words.append(data_loader.w2v_dict['w2v'].vocab[w].index)
    
print(encoded_words)

fa mai mica intensitatea iluminatului in sufragerie la 4
ba
[11165, 23, 5076, 10840, 66383, 50, 30878, 8, 129]


#### 7. Next we need to pad the sentence to the length of the longest sentence in the training set.
(weird constraint, but the motivation behind it is that each utterance has a random length which is a porblem because we would like stitch multiple utterances together into matrices and perform faster computations on them. Different lengths are an abonimation in matrices so we decided to pad them, but how long should the sentences be? If too long then we perform a lot of computations for no reason, if too short we risk loosing relevant informations from sentences since we would need to trim --> this led us to the trade off to pad every sentence to be as long as the longest one in the training set. 

In [6]:
max_len = data['max_len']
print("original sentencce length" + len(encoded_words))
encoded_padded_words = np.append(encoded_words, np.zeros((max_len - len(encoded_words),), dtype=np.int64))
data['sample_utterance'] = encoded_padded_words
print("padded sentence length" + len(encoded_padded_words))

9


#### 8. Finally we can perform the prediction using the loaded model.

In [ ]:
slots, intents = main_module.evaluate_sample(capsnet, data, sess)
